In [38]:
import os
os.environ["LANGCHAIN_PROJECT"] = "Test"

In [22]:
import pandas as pd

# QA
inputs = [
    "LangSmithの3つの主な機能は？",
    "Tracingはどういう単位で記録されますか？",
    "PharmaXではもともとどのような評価ツールを使っていましたか？"
]

outputs = [
    "TracingとEvaluationとPromptsです。",
    "Runという単位で記録されます。",
    "PromptLayerです。"
]

qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

csv_path = "./LangSmith.csv"
df.to_csv(csv_path, index=False)

In [3]:
from langsmith import Client

client = Client()
dataset_name = "LangSmith"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about LangSmith Blog Article.",
)

client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id
)

In [4]:
new_questions = [
    "LangSmithでトレースを開始するためにはどうすればいいですか？"
]

new_answers = [
    "環境変数を設定します。"
]

client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id
)

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://zenn.dev/pharmax/articles/61edc477e4de17"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
text = [p.text for p in soup.find_all("p")]
full_text = "\n".join(text)
print(full_text)

In [7]:
import openai
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(openai.Client())

In [8]:
def answer_langsmith_question_oai(inputs: dict) -> dict:
    system_msg = f"以下のコンテキストをもとに、ユーザの質問に2～3文程度で回答してください。\n\n\n{full_text}"

    messages = [{
        "role": "system",
        "content": system_msg
    }, {
        "role": "user",
        "content": inputs["question"]
    }]

    response = openai_client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")

    return {"answer": response.choices[0].message.content}

In [ ]:
answer_langsmith_question_oai({"question": "LangSmithの3つの主な機能は？"})

In [ ]:
answer_langsmith_question_oai({"question": "LangSmithでトレースを開始するためにはどうすればいいですか？"})

In [13]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

qa_evaluator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "LangSmith"

In [ ]:
experiment_results = evaluate(
    answer_langsmith_question_oai,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-langsmith-qa-gpt-3.5-turbo",
    metadata={
        "variant": "LangSmithのブログ記事をコンテキストにしたQA、gpt-3.5-turboを利用"
    }
)

In [15]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

qa_evaluator = [LangChainStringEvaluator("qa")]
dataset_name = "LangSmith"

In [ ]:
experiment_results = evaluate(
    answer_langsmith_question_oai,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-langsmith-qa-oai",
    metadata={
        "variant": "LangSmithのブログ記事をコンテキストにしたQA、gpt-4o-miniを利用"
    }
)

## カスタム評価関数

In [ ]:
from langsmith.schemas import Run, Example


# ルールベースの評価関数
# run: LLMの出力
# example: Ground TruthのQAペア
def is_answered(run: Run, example: Example) -> dict:
    # LLMの出力のこと
    student_answer = run.outputs.get("answer")

    # answerが含まれているかチェック
    if not student_answer:
        return {"key": "is_answered", "score": 0}
    else:
        return {"key": "is_answered", "score": 1}


qa_evaluator = [is_answered]
dataset_name = "LangSmith"

experiment_results = evaluate(
    answer_langsmith_question_oai,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-langsmith-qa-custom-eval-is-answered",
    metadata={
        "variant": "LangSmithのブログ記事をコンテキストにしたQA、gpt-4o-miniを利用"
    }
)

In [ ]:
from langsmith import Client

client = Client()

examples = [
    ("Ankush", "Hello Ankush"),
    ("Harrison", "Hello Harrison"),
]

dataset_name = "Hello Set"
dataset = client.create_dataset(dataset_name=dataset_name)
inputs, outputs = zip(
    *[({"input": input}, {"expected": expected}) for input, expected in examples]
)

In [ ]:
inputs

In [ ]:
outputs

In [35]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset.id
)

In [ ]:
dataset_name = "Hello Set wo Reference"
dataset = client.create_dataset(dataset_name=dataset_name)
inputs = [{"input": input} for input, _ in examples]
inputs

In [46]:
client.create_examples(
    inputs=inputs,
    dataset_id=dataset.id
)

In [ ]:
from langsmith import Client
from langsmith.evaluation import LangChainStringEvaluator, evaluate

cot_qa_evaluator = LangChainStringEvaluator("cot_qa")

evaluate(
    lambda input: "Hello " + input["input"],
    data=dataset_name,
    evaluators=[cot_qa_evaluator],
)

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

criteria_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "say_hello": "Does the submission say hello?",
        }
    }
)

evaluate(
    lambda input: "Hello " + input["input"],
    # GTがないデータセットに対しても適用できる
    data="Hello Set wo Reference",
    evaluators=[criteria_evaluator],
)

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

criteria_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "creativity": "Is this submission creative, imaginative, or novel?",
        }
    }
)

evaluate(
    lambda input: "Hello " + input["input"],
    data="Hello Set",
    evaluators=[criteria_evaluator],
)

## RAGの検索の評価

In [50]:
import pandas as pd

inputs = [
    {"question": "agent memory", "doc_txt": "agent memory has two types: short and long term"},
    {"question": "hallucinations", "doc_txt": "DBRX was pretrained on 12T tokens"},
    {"question": "DBRX content window", "doc_txt": "DBRX has a 32K token context window"},
]

# reference
# yes => relevant
# no => not relevant
outputs = [
    "yes",
    "no",
    "yes"
]

In [51]:
from langsmith import Client

client = Client()
dataset_name = "Relevance_grade"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Testing relevance grading."
)

client.create_examples(
    inputs=inputs,
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id
)